<a href="https://colab.research.google.com/github/sinjy1203/deep-learning/blob/master/mnist_camera_opencv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

mnist 데이터 학습 단계

In [0]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

import numpy as np
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0
x_train = x_train.reshape((-1, 28, 28, 1))
x_test = x_test.reshape((-1, 28, 28, 1))
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

from tensorflow.keras.models import Model
from tensorflow.keras import layers, Input

input = Input(shape=(28, 28, 1))
x = layers.Conv2D(32, 3, padding='SAME', activation='relu')(input)
x = layers.Conv2D(32, 3, padding='SAME', activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(64, 3, padding='SAME', activation='relu')(x)
x = layers.Conv2D(64, 3, padding='SAME', activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Flatten()(x)
x = layers.Dense(100, activation='relu')(x)
output = layers.Dense(10, activation='softmax')(x)
model = Model(input, output)
model.summary()

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

model.save("./mnist2_model.h5")

카메라로 숫자를 인풋받아서 예측값 출력(파이참에서)

In [0]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import load_model
import cv2 as cv

model = load_model("mnist2_model.h5")

cap = cv.VideoCapture(1)
width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
while True:
    ret, frame = cap.read()
    gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    if ret == False:
        continue
    cv.rectangle(gray_frame, (250,150), (width-250, height-150), (0,0,255))
    cv.imshow("gray", gray_frame)
    if cv.waitKey(1)%0xFF == 27:
        break
    elif cv.waitKey(1) == 32:
        img = gray_frame[150:height - 150, 250:width - 250]  #shape = 180 140
        img_resize = cv.resize(img, (28,28))
        ret, img_result = cv.threshold(img_resize, 127, 255, cv.THRESH_BINARY_INV)
        img_result_ = np.reshape(img_result, (1,28,28,1))
        print(np.argmax(model.predict(img_result_)))
        cv.imshow("mnist_mini", img_result)
        cv.imshow("mnist", img)
        cv.waitKey(0)
cap.release()
cv.destroyAllWindows()